In [1]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/RAG/'
else:
    path = '../'


# import huggingface_hub
# huggingface_hub.login()


Mounted at /content/drive


In [41]:
model_name = "beomi/Llama-3-Open-Ko-8B"

In [2]:
import pandas as pd
train =  pd.read_csv(path + 'data/'+ 'train_words.csv', index_col  = 0)
test =  pd.read_csv(path + 'data/'+ 'test_one.csv', index_col  = 0)

In [3]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "beomi/Llama-3-Open-Ko-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### 데이터셋 불러오기


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# tokenizer =  AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map = "auto",)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

In [6]:
from datasets import DatasetDict, Dataset
train_dataset = DatasetDict()
test_dataset = DatasetDict()


train_dataset = Dataset.from_pandas(train)
test_dataset['test'] = Dataset.from_pandas(test)

In [ ]:

alpaca_prompt = """
### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["사고원인"]
    outputs      = examples["재발방지대책 및 향후조치계획"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
    pass
dataset_preprocessed = train_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/23422 [00:00<?, ? examples/s]

In [ ]:

# instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# def formatting_prompts_func(examples):
#     global instruction
#     instructions = [instruction for _ in range(6614)]
#     inputs       = examples["사고원인"]
#     outputs      = examples["재발방지대책 및 향후조치계획"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         messages = [
#             {"role": "system", "content": f"{instruction}"},
#             {"role": "user", "content": f"{input}"},
#             {"role": "assistant", "content": f"{output}"},
#             ]

#         text = tokenizer.apply_chat_template(messages, tokenize=False)
#         text += '<|end_of_text|>'
#         texts.append(text[17:])
#     return { "text" : texts, }
#     pass
# dataset_preprocessed = train_dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input}"},
#     {"role": "assistant", "content": f"{output}"},
#     ]

# text = tokenizer.apply_chat_template(messages, tokenize=False)
# text += '<|end_of_text|>'

In [ ]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

In [ ]:
tokenizer.eos_token

In [ ]:
# text[17:]

In [ ]:
# EOS_TOKEN

In [ ]:
# dataset_preprocessed

In [ ]:
# instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
# input = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
# output = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'

In [ ]:
# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input}"},
#     {"role": "assistant", "content": f"{output}"},
#     ]

# text = tokenizer.apply_chat_template(messages, tokenize=False)
# tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt")
# tokenized['input_ids'][0]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_preprocessed,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/23422 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/23422 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/23422 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/23422 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(trainer.train_dataset, batch_size=trainer.args.per_device_train_batch_size, shuffle=True)
# batch = next(iter(dataloader))


In [ ]:
tokenizer.decode(trainer.train_dataset['input_ids'][0])

'<|begin_of_text|>\n### Input:\n고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치가 미흡하였음.\n\n### Response:\n고소작업 시 추락 위험이 있는 부위에 안전장비 설치.<|end_of_text|>'

In [ ]:
# tokenizer.add_special_tokens({'pad_token': '<|reserved_special_token_0|>'})


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 23,422 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.038200
2,2.136800
3,1.872800
4,2.057900
5,2.047000
6,2.126800
7,1.967500
8,1.830000
9,2.297600
10,1.993700


In [ ]:
alpaca_prompt_test = """
### Input:
{}

### Response:
"""

# instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func_test(examples):
    # global instruction
    # instructions = [instruction for _ in range(6614)]
    input       = examples["사고원인"]
    texts = []
    text = alpaca_prompt_test.format(input)
    texts.append(text)
    return { "text" : texts, }
    pass

In [ ]:
 input = test.iloc[4]
 test_text = formatting_prompts_func_test(input)['text'][0]
 test_text


'\n### Input:\n점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생\n\n### Response:\n'

In [ ]:
# instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
# input = test.iloc[3].item()
# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input}"},
#     {"role": "assistant", "content": ""},
#     ]

# test_text = tokenizer.apply_chat_template(messages, tokenize=False)

In [ ]:
# print(test_text)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer([test_text], return_tensors = "pt").to("cuda")
# inputs
from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
ouput = model.generate(**inputs, max_new_tokens = 256, eos_token_id=tokenizer.eos_token_id, early_stopping=True,)

In [ ]:
tokenizer.decode(ouput[0])

'<|begin_of_text|>\n### Input:\n점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생\n\n### Response:\n작업자 안전교육 실시 및 안전교육 이수 확인을 통한 재발 방지 대책.<|end_of_text|>'

In [ ]:
model.save_pretrained(path + 'second_model/Llama-3-Open-Ko-8B')
tokenizer.save_pretrained(path + 'second_model/Llama-3-Open-Ko-8B')

('/content/drive/MyDrive/RAG/second_model/Llama-3-Open-Ko-8B/tokenizer_config.json',
 '/content/drive/MyDrive/RAG/second_model/Llama-3-Open-Ko-8B/special_tokens_map.json',
 '/content/drive/MyDrive/RAG/second_model/Llama-3-Open-Ko-8B/tokenizer.json')

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = path + 'second_model/Llama-3-Open-Ko-8B',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
alpaca_prompt_test = """
### Input:
{}
### Response:
"""
temp = True
def formatting_prompts_func_submit(examples):
    inputs       = examples["사고원인"]
    text = alpaca_prompt_test.format(inputs)
    return { "text" : text, }
    pass

submit_dataset =  test_dataset['test'].map(formatting_prompts_func_submit)

Map:   0%|          | 0/964 [00:00<?, ? examples/s]

In [8]:
train_submit_dataset =  train_dataset.map(formatting_prompts_func_submit)

Map:   0%|          | 0/23422 [00:00<?, ? examples/s]

In [9]:
train_submit_dataset

Dataset({
    features: ['사고원인', '재발방지대책 및 향후조치계획', '__index_level_0__', 'text'],
    num_rows: 23422
})

In [ ]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# # alpaca_prompt = You MUST copy from above!

# inputs = tokenizer(submit_dataset['text'],padding=True, truncation=True, return_tensors = "pt").to("cuda")
# # # inputs
# ouput = model.generate(**inputs, max_new_tokens = 256, eos_token_id=tokenizer.eos_token_id, early_stopping=True,)

In [ ]:
# inputs.to('cpu')
# del inputs
# torch.cuda.empty_cache()

In [11]:
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader

FastLanguageModel.for_inference(model) # Enable native 2x faster inference


batch_size = 1  # 원하는 배치 사이즈 설정
# texts = submit_dataset['text']  # 텍스트 데이터 리스트
texts = train_submit_dataset['text']  # 트레인 데이터 시험용

# DataLoader를 사용해 배치 단위로 데이터를 가져옵니다.
data_loader = DataLoader(texts[:200], batch_size=batch_size, shuffle = False)
save_idx = 0

all_outputs = []
decoded_outputs = []
for batch_texts in tqdm(data_loader):
    # 각 배치마다 토크나이징: 패딩과 트렁케이션 적용
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to("cuda")
    # 모델의 generate 함수 호출
    batch_outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=True,
        use_cache=True,
        repetition_penalty = 1.1
    )
    all_outputs.append(batch_outputs)
    batch_outputs.to('cpu')
    decoded_outputs += [tokenizer.decode(batch_output, skip_special_tokens=True) for batch_output in batch_outputs]
    del batch_outputs
    torch.cuda.empty_cache()


  0%|          | 0/200 [00:00<?, ?it/s]

In [13]:
decoded_outputs[:5]

[]

In [12]:
df = pd.DataFrame(decoded_outputs, columns=['values'])
df.to_csv(path + 'data/'+  'train_words_second.csv')

In [14]:
# pd.read_csv(path + 'data/'+  'train_words_second.csv').head()

,Unnamed: 0,values
0,0,"\n### Input:\n고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, ..."
1,1,\n### Input:\n부주의\n### Response:\n안전교육 실시 및 안전...
2,2,\n### Input:\n3층 슬라브 작업시 이동중 미끄러짐\n### Respon...
3,3,\n### Input:\n교각 기초철근 조립 중 강한 바람에 의해 기둥측 주철근이 ...
4,4,\n### Input:\n근로자의 부주의\n### Response:\n작업 전 안전...


In [ ]:
all_outputs[0].shape
all_outputs[1].shape

torch.Size([8, 360])

In [ ]:
decoded_outputs

['\n### Input:\n펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨\n### Response:\n펌프카 아웃트리거 바닥 고임목을 3단으로 보강하고, 펌프카 붐대호스 최대한 펼치지 않도록 교육 실시.',
 '\n### Input:\n작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용\n### Response:\n작업자 안전교육 실시 및 보안면 착용 철저 교육 실시.',
 '\n### Input:\n작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐\n### Response:\n작업자 안전교육 실시 및 안전교육 이수 확인을 통한 재발 방지 대책.',
 '\n### Input:\n작업 발판 위 벽돌 잔재를 밟고 넘어짐\n### Response:\n작업 발판 위 잔재 제거 및 작업자 안전교육 실시.']

In [ ]:
# inputs = tokenizer(super_test_text, return_tensors = "pt").to("cuda")

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
inputs

{'input_ids': tensor([[128000,    198,  14711,   5688,    512, 101838, 102612,  77437, 111636,
         107472, 102426,  30426, 121002,  26799,  86503, 109627,  17835,  59777,
          24486,  58935, 105316,  56154,  35495, 113610,    271,  14711,   6075,
            512],
        [128000,    198,  14711,   5688,    512, 101838, 102612,  77437, 111636,
         107472, 102426,  30426, 121002,  26799,  86503, 109627,  17835,  59777,
          24486,  58935, 105316,  56154,  35495, 113610,    271,  14711,   6075,
            512]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]], device='cuda:0')}

In [ ]:
import pandas as pd
import os

my_answer =  pd.read_csv(path + 'data/' + 'train_words0.csv', index_col=0)
# my_answer

In [ ]:
my_answer_response =  my_answer.map(lambda x: x.split('### Response:\n')[1])

In [ ]:
my_answer_response

,values
0,"펌프카 아웃트리거 바닥 고임목을 3단으로 보강하고, 펌프카 붐대호스 최대한 펼치지 ..."
1,작업자 안전교육 실시 및 보안면 착용 철저 교육 실시.
2,작업자 안전교육 실시 및 안전교육 이행 여부 확인.
3,작업 발판 위 잔재 제거 및 작업자 안전교육 실시.
4,작업자 안전교육 실시 및 안전교육 이행 여부 확인을 통한 재발 방지 대책.
...,...
959,석재의 파손을 방지하기 위한 조치 계획.
960,작업 전 이동통로 확보 철저 및 안전교육 실시.
961,작업자 안전교육 실시 및 안전교육 이행 여부 확인.
962,작업 전 안전교육 실시와 안전장비 착용 철저한 관리감독 실시.


In [ ]:
sub_samp =  pd.read_csv( path + 'data/' + 'sample_submission.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = list(my_answer_response.iloc[:, 0])

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-sts')
model = AutoModel.from_pretrained('jhgan/ko-sbert-sts').to('cuda')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to('cuda')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
tensor_df =  pd.DataFrame(sentence_embeddings.to('cpu').numpy())
sample_parsed =  pd.concat( [sub_samp.iloc[:, :2], tensor_df], axis = 1 )

In [ ]:
sample_parsed.columns = sub_samp.columns
sample_parsed.iloc[:5, :5]

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2
0,TEST_000,안전교육 실시,-0.234351,0.255808,-0.200545
1,TEST_001,안전교육 실시,-1.233831,0.844816,-0.313282
2,TEST_002,안전교육 실시,-0.487280,0.705935,-0.284927
3,TEST_003,안전교육 실시,-0.215420,0.706537,-0.413395
4,TEST_004,안전교육 실시,-0.091436,0.462474,0.473014


In [ ]:
sample_parsed.to_csv(path + 'data/' + '두번째_제출_2.csv', index=None)

In [ ]:
pd.read_csv(path + 'data/' + '두번째_제출_2.csv')

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,안전교육 실시,-0.234351,0.255808,-0.200545,-0.039682,-0.305646,0.682476,1.939487,0.423817,...,0.169860,0.614356,0.129557,1.242781,0.388476,0.711620,-0.429775,-0.714544,-0.097917,0.228594
1,TEST_001,안전교육 실시,-1.233831,0.844816,-0.313282,-0.033262,-0.570466,0.441022,0.560929,0.228697,...,0.370556,1.997712,0.933945,1.740040,1.255820,-0.120332,0.218243,-0.338179,0.464741,-0.029279
2,TEST_002,안전교육 실시,-0.487281,0.705935,-0.284927,0.417743,-0.719636,0.210797,0.441540,-0.179384,...,0.714164,2.487258,1.189492,2.193630,1.227424,-0.255178,-0.231713,0.688298,0.821369,-0.171505
3,TEST_003,안전교육 실시,-0.215420,0.706537,-0.413395,1.077453,-0.782626,0.897697,0.946338,0.286951,...,0.234308,0.877054,0.173862,2.354648,0.631869,0.441496,0.199307,-0.426155,1.113010,0.217471
4,TEST_004,안전교육 실시,-0.091436,0.462474,0.473014,0.425276,-0.317348,-0.044728,0.196677,0.029001,...,0.471844,2.108704,1.546166,2.814868,0.902282,-0.977493,-0.219824,0.390175,1.173428,-0.167014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,TEST_959,안전교육 실시,0.975186,0.314578,-0.485058,1.324071,0.742440,0.927296,-0.056763,-0.225125,...,1.377117,-0.148667,-0.055105,1.964376,1.012007,0.701201,0.542704,-0.391002,1.071752,0.486827
960,TEST_960,안전교육 실시,-0.739075,0.477664,-0.660712,0.783448,-1.276444,1.198082,0.629846,0.001739,...,0.048355,1.979565,0.410719,2.500354,1.157479,-0.160430,-0.856274,-0.125738,0.603135,-0.061463
961,TEST_961,안전교육 실시,-0.487281,0.705935,-0.284927,0.417743,-0.719636,0.210797,0.441540,-0.179384,...,0.714164,2.487258,1.189492,2.193630,1.227424,-0.255178,-0.231713,0.688298,0.821369,-0.171505
962,TEST_962,안전교육 실시,-0.560022,0.465145,-0.296587,0.667906,-0.385411,0.738884,0.519610,-0.037035,...,0.367917,1.733710,0.885684,2.146897,1.066704,-0.863159,-0.137085,-0.377530,0.317326,-0.159975
